## 1. Importar Librerias

In [ ]:
#pip install rapidfuzz

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import numpy as np
from matplotlib import cm
import random
import glob
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler
import joblib
import re
import unicodedata

from rapidfuzz import fuzz, process #limpieza datos
from itertools import combinations
from collections import Counter
from scipy.sparse import csr_matrix, coo_matrix

import unidecode
def remove_accents(a):
    return unidecode.unidecode(a)

##  2. Carga y Procesamiento de Datos

En este proceso, se cargaran bases de licitaciones públicas en Chile entre los años 2014-2018.Para esto, se utilizará las licitaciones, la base de sectores para identificar a que sector pertenece la empresa (municipalidad, salud, etc) y por último la base de rubros que nos dará más información sobre la categoría del producto.  

### 2.1 Cargar licitaciones

In [ ]:
# Definir la ruta donde se encuentran los archivos de licitaciones
path = 'C:/Users/SCL_SERVIDOR/Desktop/Capstone Project'  # Cambia esto si tus archivos están en otra carpeta

# Definir el patrón de archivos
pattern = 'licitaciones_*.csv'

# Usar glob para obtener la lista de archivos que coinciden con el patrón
all_files = glob.glob(os.path.join(path, pattern))

# Verificar los archivos encontrados
print(f"Archivos encontrados: {all_files}")

In [ ]:
# Lista para almacenar cada DataFrame anual
list_of_dfs = []

for file in all_files:
    # Extraer el año del nombre del archivo
    basename = os.path.basename(file)  # Ejemplo: 'licitaciones_2014.csv'
    year = basename.split('_')[1].split('.')[0]
    
    # Leer el archivo CSV
    df = pd.read_csv(file, sep=',', encoding='latin-1', decimal='.')
    
    # Añadir una columna para el año
    df['Año'] = int(year)
    
    # Agregar el DataFrame a la lista
    list_of_dfs.append(df)

# Concatenar todos los DataFrames en uno solo
lic_all_years = pd.concat(list_of_dfs, ignore_index=True)

# Verificar la concatenación
print(f"Total de registros combinados: {lic_all_years.shape[0]}")

### 2.2 Fusión con sectores

In [ ]:
ruta='C:/Users/chaco/OneDrive/Escritorio/Magister Data Science/Capstone Proyect/Cod'
archivos_en_ruta=os.listdir(ruta)
print(archivos_en_ruta)

In [ ]:
archivo=os.path.join(ruta,'sectores.csv')
try:
    sectores=pd.read_csv(archivo,sep=";")
    print("Archivo cargado correctamente.")
except FileNotFoundError:
    print(f"El archivo '{archivo}'no se encuentra")
except Exception as e:
    print(f"Error al cargar el archivo: {e}")

In [ ]:
# Asegurarse de que las columnas clave están en el mismo formato
lic_all_years['NombreOrganismo'] = lic_all_years['NombreOrganismo'].astype(str).str.strip()
sectores['Nombre de la institución'] = sectores['Nombre de la institución'].astype(str).str.strip()

# Fusionar con sector 
lic_all_years = pd.merge(
    lic_all_years,
    sectores[['Nombre de la institución','Sector']],
    how='left',
    left_on="NombreOrganismo",
    right_on='Nombre de la institución'
)

lic_all_years.drop(columns=["Nombre de la institución"],inplace=True)

### 2.3 Fusión con Rubros


In [ ]:
archivo=os.path.join(ruta,'rubros_onu.xlsx')
try:
    rubros_onu=pd.read_excel(archivo)
    print("Archivo cargado correctamente.")
except FileNotFoundError:
    print(f"El archivo '{archivo}'no se encuentra")
except Exception as e:
    print(f"Error al cargar el archivo: {e}")

In [ ]:
# Asegurarse de que las columnas clave están en el mismo formato
lic_all_years['CodigoProductoONU'] = lic_all_years['CodigoProductoONU'].astype(str).str.strip()
rubros_onu['CodigoProductoONU'] = rubros_onu['CodigoProductoONU'].astype(str).str.strip()

# Fusionar con rubros ONU
lic_all_years = pd.merge(
    lic_all_years,
    rubros_onu[['CodigoProductoONU','NombreProducto','RubroN2']],
    how='left',
    on='CodigoProductoONU'
)

### 2.4 Guardar y Cargar Data Unificada (2014-2023)

In [ ]:
#ruta = 'C:/Users/SCL_SERVIDOR/Desktop/Capstone Project/lic_combinadas_2014_2018.csv'
#lic_all_years.to_csv(ruta, index=False, encoding='utf-8-sig')

In [ ]:
lic_all_years = pd.read_csv('lic_combinadas_2014_2023.csv')

## 3. Limpieza de Datos

Se redujo la cantidad de proveedores de 175278 a 155680 lo que nos da un total de 19598 proveedores menos.

In [ ]:
# Eliminar filas donde 'NombreProveedor' sea vacío (""), guion ("-"), punto (".") o nulo (NaN)
lic_all_years_clean = lic_all_years[~lic_all_years["NombreProveedor"].isin(["-", ".","---------","- - -"," ","--","0","000000","1"]) 
                    & lic_all_years["NombreProveedor"].notna()]

In [ ]:
lic_all_years_clean["NombreProveedor"].isna().sum()

In [ ]:
lic_all_years["NombreProveedor"].nunique(),lic_all_years_clean["NombreProveedor"].nunique()

In [ ]:
# Función para limpiar y estandarizar nombres
def limpiar_nombre(nombre):
    if pd.isna(nombre):  # Verifica si el valor es NaN
        return np.nan
    # Eliminar tildes
    nombre = unicodedata.normalize('NFD', nombre).encode('ascii', 'ignore').decode('utf-8')
    # Convertir a minúsculas
    nombre = nombre.lower()
    # Eliminar caracteres no alfabéticos y no numéricos (excepto espacios)
    nombre = re.sub(r'[^a-zA-Z0-9\s]', '', nombre)
    # Eliminar palabras como "spa", "ltda", etc.
    nombre = re.sub(r'\b(spa|ltda|sac|sa|sas|corp|inc|company|co|com|limit|limitada|limita|anónima)\b', '', nombre)
    # Eliminar múltiples espacios y dejar solo uno
    nombre = re.sub(r'\s+', ' ', nombre).strip()
    return nombre


# Crear una nueva columna para los nombres originales
lic_all_years_clean['NombreProveedorOriginal'] = lic_all_years_clean['NombreProveedor']

# Limpiar y crear la columna de nombres limpios
lic_all_years_clean['NombreProveedorLimpio'] = lic_all_years_clean['NombreProveedor'].apply(limpiar_nombre)

# Comparar nombres originales vs limpios
comparacion_nombres = lic_all_years_clean[['NombreProveedorOriginal', 'NombreProveedorLimpio']]

In [ ]:
# Eliminar filas donde 'NombreProveedorLimpio' es vacío ("") después de la limpieza
lic_all_years_clean = lic_all_years_clean[lic_all_years_clean['NombreProveedorLimpio'] != ""]

In [ ]:
# Verificar el cambio, mostrando los primeros registros
lic_all_years_clean["NombreProveedorLimpio"].isna().sum(),lic_all_years_clean["NombreProveedorLimpio"].nunique()

### 3.1 Filtrar datos del 2014-2018

In [ ]:
# Filtrar la data por los años 2014 a 2018
lic_all_years_clean_2014_2018 = lic_all_years_clean[(lic_all_years_clean['Año'] >= 2014) & (lic_all_years_clean['Año'] <= 2018)]
lic_all_years_clean_2019_2023 = lic_all_years_clean[(lic_all_years_clean['Año'] >= 2019) & (lic_all_years_clean['Año'] <= 2023)]

# Mostrar los primeros registros del DataFrame filtrado
lic_all_years_clean_2019_2023["Año"].unique()

## 4. Cálculo VCR
- No elimines duplicados indiscriminadamente: Cada registro puede representar una participación válida y única.
- Asegúrate de que los datos reflejen la realidad de las participaciones: Esto garantizará que el VCR y otras métricas sean precisas.
- Adapta tu análisis a la naturaleza de tus datos: Considera todas las variables relevantes para tu contexto.

In [ ]:
licitacion_col = 'Codigo'
proveedor_col = 'NombreProveedorLimpio'
rubro_col = 'RubroN2'
producto_col = 'CodigoProductoONU'

# Eliminar duplicados basados en licitación, proveedor, rubro y producto
#lic_all_years_unique = lic_all_years_clean.drop_duplicates(subset=[licitacion_col, proveedor_col, rubro_col, producto_col])
lic_all_years_unique_2014_2018 = lic_all_years_clean_2014_2018

# Paso 1: Calcular el número total de participaciones en todos los rubros y proveedores
participaciones_totales = len(lic_all_years_unique_2014_2018)

# Paso 2: Calcular las participaciones únicas de cada proveedor en cada rubro
participaciones_proveedor_rubro = lic_all_years_unique_2014_2018.groupby([proveedor_col, rubro_col]).size()

# Paso 3: Calcular las participaciones totales por proveedor
participaciones_totales_proveedor = lic_all_years_unique_2014_2018[proveedor_col].value_counts()

# Paso 4: Calcular las participaciones totales por rubro
participaciones_totales_rubro = lic_all_years_unique_2014_2018[rubro_col].value_counts()

# Combinar los cálculos en un único DataFrame
vcr_df = participaciones_proveedor_rubro.reset_index(name='participaciones_proveedor_rubro')
vcr_df['participaciones_totales_proveedor'] = vcr_df[proveedor_col].map(participaciones_totales_proveedor)
vcr_df['participaciones_totales_rubro'] = vcr_df[rubro_col].map(participaciones_totales_rubro)

# Calcular el VCR utilizando la fórmula vectorizada
vcr_df['VCR'] = (vcr_df['participaciones_proveedor_rubro'] / vcr_df['participaciones_totales_proveedor']) / \
                (vcr_df['participaciones_totales_rubro'] / participaciones_totales)

# Filtrar proveedores con VCR >= 1 para indicar ventaja comparativa
vcr_df['ventaja_comparativa'] = vcr_df['VCR']>=1

In [ ]:
vcr_df["ventaja_comparativa"].value_counts()

In [ ]:
vcr_df.head()

## 5. Calculo de Matriz Especialización

In [ ]:
# Paso 1: Filtrar los proveedores con ventaja comparativa
especializacion_df = vcr_df[vcr_df['ventaja_comparativa']][[proveedor_col, rubro_col]]

# Paso 2: Crear índices para proveedores y rubros
proveedores = especializacion_df[proveedor_col].unique()
rubros = especializacion_df[rubro_col].unique()

proveedor_idx = {proveedor: idx for idx, proveedor in enumerate(proveedores)}
rubro_idx = {rubro: idx for idx, rubro in enumerate(rubros)}

# Mapear los nombres a índices
especializacion_df['proveedor_idx'] = especializacion_df[proveedor_col].map(proveedor_idx)
especializacion_df['rubro_idx'] = especializacion_df[rubro_col].map(rubro_idx)

# Verificar que no hay valores NaN en los índices
assert not especializacion_df['proveedor_idx'].isnull().any(), "Hay proveedores sin índice."
assert not especializacion_df['rubro_idx'].isnull().any(), "Hay rubros sin índice."

In [ ]:
# Asegurarse de que los índices son enteros
especializacion_df['proveedor_idx'] = especializacion_df['proveedor_idx'].astype(int)
especializacion_df['rubro_idx'] = especializacion_df['rubro_idx'].astype(int)

# Paso 3: Construir la matriz de especialización dispersa
data = np.ones(len(especializacion_df), dtype=np.int32)  # Cambiado a np.int32
rows = especializacion_df['rubro_idx'].values
cols = especializacion_df['proveedor_idx'].values

matriz_especializacion_sparse = csr_matrix((data, (rows, cols)), shape=(len(rubros), len(proveedores)))

# Verificar si hay valores negativos en matriz_especializacion_sparse
if (matriz_especializacion_sparse.data < 0).any():
    print("Advertencia: Se encontraron valores negativos en matriz_especializacion_sparse")

# Paso 4: Calcular el producto matricial disperso
producto_sparse = matriz_especializacion_sparse @ matriz_especializacion_sparse.T

# Verificar si hay valores negativos en producto_sparse
if (producto_sparse.data < 0).any():
    print("Advertencia: Se encontraron valores negativos en producto_sparse")

# Paso 5: Calcular la suma de proveedores por rubro
suma_por_rubro = np.array(matriz_especializacion_sparse.sum(axis=1)).flatten()

## 6. Cálculo de Proximdiad


In [ ]:
# Paso 6: Calcular la matriz de proximidad
producto_coo = producto_sparse.tocoo()

max_sumas = np.maximum(suma_por_rubro[producto_coo.row], suma_por_rubro[producto_coo.col])

# Evitar divisiones por cero
max_sumas[max_sumas == 0] = np.finfo(float).eps

# Calcular la proximidad
proximidad_data = producto_coo.data / max_sumas

# Verificar si hay valores negativos en proximidad_data
if (proximidad_data < 0).any():
    print("Advertencia: Se encontraron valores negativos en proximidad_data")

# Crear la matriz dispersa de proximidad
proximidad_sparse = coo_matrix((proximidad_data, (producto_coo.row, producto_coo.col)), shape=producto_sparse.shape)

# Opcional: Convertir a DataFrame si es manejable en memoria
idx_to_rubro = {idx: rubro for rubro, idx in rubro_idx.items()}
rubro_labels = [idx_to_rubro[idx] for idx in range(len(rubros))]

proximidad_df = pd.DataFrame.sparse.from_spmatrix(proximidad_sparse, index=rubro_labels, columns=rubro_labels)

In [ ]:
proximidad_df.head()

In [ ]:
proximidad_df.columns

In [ ]:
print(f"Valor mínimo de proximidad: {proximidad_data.min()}")
print(f"Valor máximo de proximidad: {proximidad_data.max()}")

## 7. Red de Rubros 

In [ ]:
# Crear un grafo desde la matriz de proximidad
G = nx.from_pandas_adjacency(proximidad_df)

In [ ]:
nx.write_gexf(G,"GrafoProximidadRubros.gexf")

In [ ]:
# Mostrar información básica del grafo
print(f"Grafo creado con {G.number_of_nodes()} nodos y {G.number_of_edges()} aristas.")

# Opcional: Guardar el grafo en un archivo para visualización (formato GraphML o GEXF)
nx.write_gexf(G, "grafo_proximidad.gexf")

In [ ]:
# Crear un diccionario para almacenar las métricas
metrics = {}

# Métricas básicas
metrics['weighted_degree'] = dict(G.degree(weight='weight'))  # Grado ponderado
metrics['betweenness_centrality'] = nx.betweenness_centrality(G, weight='weight')  # Centralidad de intermediación
metrics['closeness_centrality'] = nx.closeness_centrality(G)  # Centralidad de cercanía
metrics['pagerank'] = nx.pagerank(G, weight='weight')  # PageRank
metrics['clustering_coefficient'] = nx.clustering(G, weight='weight')  # Coeficiente de agrupamiento

# Crear un DataFrame con todas las métricas
metrics_df = pd.DataFrame(metrics)

In [ ]:
# Resetear el índice para incluir los nodos como una columna
metrics_df.reset_index(inplace=True)

# Renombrar la columna del índice como "Rubro"
metrics_df.rename(columns={'index': 'Rubro'}, inplace=True)

In [ ]:
# Mostrar las primeras filas del DataFrame
metrics_df.head()

## 8. Cálculo de Density Relatedness 
Para cada proveedor y rubro no especializado, calcularemos la density relatedness como el promedio ponderado de la proximidad a los rubros en los que el proveedor ya está especializado.

In [ ]:
# Preparar matrices y diccionarios
matriz_especializacion_csr = matriz_especializacion_sparse.tocsr()
proximidad_csr = proximidad_sparse.tocsr()
todos_los_rubros = set(range(len(rubros)))
idx_to_proveedor = {idx: proveedor for proveedor, idx in proveedor_idx.items()}
idx_to_rubro = {idx: rubro for rubro, idx in rubro_idx.items()}

In [ ]:
# Inicializar listas para resultados
lista_proveedores = []
lista_rubros_potenciales = []
lista_density_relatedness = []

# Obtener la diagonal completa de proximidad_csr una sola vez
diagonal_proximidad = proximidad_csr.diagonal()

# Calcular el portafolio de cada proveedor y la density relatedness
for proveedor_idx_actual in range(matriz_especializacion_csr.shape[1]):
    rubros_con_ventaja = matriz_especializacion_csr[:, proveedor_idx_actual].nonzero()[0]
    rubros_con_ventaja_set = set(rubros_con_ventaja)
    rubros_potenciales = np.array(list(todos_los_rubros - rubros_con_ventaja_set))
    if rubros_potenciales.size == 0:
        continue

    # Obtener las proximidades con el portafolio del proveedor
    proximidades_con_portafolio = proximidad_csr[rubros_potenciales[:, None], rubros_con_ventaja].toarray()

    # Sumar las proximidades con el portafolio para cada rubro potencial
    suma_proximidades_portafolio = proximidades_con_portafolio.sum(axis=1)

    # Obtener las proximidades totales del rubro potencial
    proximidades_totales = proximidad_csr[rubros_potenciales].toarray()

    # Sumar todas las proximidades excepto la del rubro consigo mismo (diagonal)
    diagonales_rubros_potenciales = diagonal_proximidad[rubros_potenciales]
    suma_proximidades_totales = proximidades_totales.sum(axis=1) - diagonales_rubros_potenciales

    # Evitar divisiones por cero
    suma_proximidades_totales[suma_proximidades_totales == 0] = np.finfo(float).eps

    # Calcular la Density Relatedness
    density = suma_proximidades_portafolio / suma_proximidades_totales

    # Almacenar los resultados
    proveedores_repetidos = [idx_to_proveedor[proveedor_idx_actual]] * len(rubros_potenciales)
    rubros_potenciales_nombres = [idx_to_rubro[idx] for idx in rubros_potenciales]
    lista_proveedores.extend(proveedores_repetidos)
    lista_rubros_potenciales.extend(rubros_potenciales_nombres)
    lista_density_relatedness.extend(density)

In [ ]:
# Crear el DataFrame final
density_df = pd.DataFrame({
    'NombreProveedorLimpio': lista_proveedores,
    'RubroPotencial': lista_rubros_potenciales,
    'DensityRelatedness': lista_density_relatedness
})

In [ ]:
density_df.head()

## 9. Enriquecer Caracteristicas del DataFrame

### 9.1 Caracteristicas Proveedor

In [ ]:
import pandas as pd

# Supongamos que 'lic_all_years_clean' es tu DataFrame original
# Primero, creamos una copia del DataFrame para trabajar con ella
df = lic_all_years_clean_2014_2018.copy()

# Filtrar los montos adjudicados
df['MontoTotalAdjudicado'] = df['MontoLineaAdjudica'].where((df['MontoLineaAdjudica'] >= 1) & (df['MontoLineaAdjudica'] <= 10**12))

# Crear una columna que indique si la oferta fue seleccionada
df['Adjudicada'] = df['Oferta seleccionada'] == 'Seleccionada'

# Total adjudicaciones proveedor (solo las adjudicadas)
df['TotalAdjudicacionesProveedor'] = df.groupby('NombreProveedorLimpio')['Adjudicada'].transform('sum')

# Total participaciones proveedor
df['TotalParticipacionesProveedor'] = df.groupby('NombreProveedorLimpio')['Codigo'].transform('size')

# % Adjudicación proveedor
df['%AdjudicacionProveedor'] = (df['TotalAdjudicacionesProveedor'] / df['TotalParticipacionesProveedor']) * 100

# Total Monto adjudicado proveedor (filtrado)
df['TotalMontoAdjudicadoProveedor'] = df.groupby('NombreProveedorLimpio')['MontoTotalAdjudicado'].transform('sum')

# Total Monto promedio adjudicado proveedor (filtrado)
df['TotalMontoPromedioAdjudicadoProveedor'] = df.groupby('NombreProveedorLimpio')['MontoTotalAdjudicado'].transform('mean')

# Numero de rubros con ventaja
df['NumeroRubrosConVentaja'] = df.groupby('NombreProveedorLimpio')['RubroN2'].transform(lambda x: x.nunique())

# Total monto adjudicado rubro (filtrado)
df['TotalMontoAdjudicadoRubro'] = df.groupby('RubroN2')['MontoTotalAdjudicado'].transform('sum')

# Total monto promedio adjudicado rubro (filtrado)
df['TotalMontoPromedioAdjudicadoRubro'] = df.groupby('RubroN2')['MontoTotalAdjudicado'].transform('mean')

# Numero de proveedores rubro
df['NumeroProveedoresRubro'] = df.groupby('RubroN2')['NombreProveedorLimpio'].transform(lambda x: x.nunique())

In [ ]:
# Eliminar duplicados en df para las claves de unión
df_proveedor = df[['NombreProveedorLimpio', 'TotalAdjudicacionesProveedor', 
                   'TotalParticipacionesProveedor', '%AdjudicacionProveedor', 
                   'TotalMontoAdjudicadoProveedor', 
                   'TotalMontoPromedioAdjudicadoProveedor', 
                   'NumeroRubrosConVentaja']].drop_duplicates()

df_rubro = df[['RubroN2', 'TotalMontoAdjudicadoRubro', 
                'TotalMontoPromedioAdjudicadoRubro', 
                'NumeroProveedoresRubro']].drop_duplicates()

# Realizar el merge para las características del proveedor
density_df = density_df.merge(df_proveedor, left_on='NombreProveedorLimpio', right_on='NombreProveedorLimpio', how='left')

# Realizar el merge para las características del rubro
density_df = density_df.merge(df_rubro, left_on='RubroPotencial', right_on='RubroN2', how='left')

# Eliminar las columnas que ya existen en density_df
columnas_a_eliminar = ['RubroN2']  # Agrega más columnas si es necesario
density_df = density_df.loc[:, ~density_df.columns.isin(columnas_a_eliminar)]

# Mostrar el DataFrame resultante
density_df.head()

### 9.2 Unir con metricas de Red

In [ ]:
density_df.shape

In [ ]:
# Realizar el merge para las características del rubro
density_df = density_df.merge(metrics_df, left_on='RubroPotencial', right_on='Rubro', how='left')

# Eliminar las columnas que ya existen en density_df
columnas_a_eliminar_red = ['Rubro']  # Agrega más columnas si es necesario
density_df = density_df.loc[:, ~density_df.columns.isin(columnas_a_eliminar_red)]

In [ ]:
density_df.columns

## 10. Cargar y preprocesar datos de 2019 al 2023 [Generar Etiqueta]

In [ ]:
lic_all_years_clean_2019_2023["Año"].unique()

In [ ]:
# Paso 1: Calcular el número total de participaciones en 2019-2023
participaciones_totales_2019_2023 = len(lic_all_years_clean_2019_2023)

# Paso 2: Calcular las participaciones únicas de cada proveedor en cada rubro
participaciones_proveedor_rubro_2019_2023 = lic_all_years_clean_2019_2023.groupby([proveedor_col, rubro_col]).size()

# Paso 3: Calcular las participaciones totales por proveedor
participaciones_totales_proveedor_2019_2023 = lic_all_years_clean_2019_2023[proveedor_col].value_counts()

# Paso 4: Calcular las participaciones totales por rubro
participaciones_totales_rubro_2019_2023 = lic_all_years_clean_2019_2023[rubro_col].value_counts()

# Combinar los cálculos en un DataFrame
vcr_df_2019_2023 = participaciones_proveedor_rubro_2019_2023.reset_index(name='participaciones_proveedor_rubro')
vcr_df_2019_2023['participaciones_totales_proveedor'] = vcr_df_2019_2023[proveedor_col].map(participaciones_totales_proveedor_2019_2023)
vcr_df_2019_2023['participaciones_totales_rubro'] = vcr_df_2019_2023[rubro_col].map(participaciones_totales_rubro_2019_2023)

# Calcular el VCR
vcr_df_2019_2023['VCR'] = (vcr_df_2019_2023['participaciones_proveedor_rubro'] / vcr_df_2019_2023['participaciones_totales_proveedor']) / \
                           (vcr_df_2019_2023['participaciones_totales_rubro'] / participaciones_totales_2019_2023)

# Determinar si tiene ventaja comparativa
vcr_df_2019_2023['ventaja_comparativa'] = vcr_df_2019_2023['VCR'] >= 1

In [ ]:
vcr_df_2019_2023["ventaja_comparativa"].value_counts()

In [ ]:
vcr_2019_2023_exito = vcr_df_2019_2023[vcr_df_2019_2023['ventaja_comparativa']][[proveedor_col, rubro_col]].copy()
vcr_2019_2023_exito['Exito'] = 1

In [ ]:
vcr_2019_2023_exito.rename(columns={"RubroN2":"RubroPotencial"},inplace=True)

In [ ]:
vcr_2019_2023_exito.columns

In [ ]:
# Realizar el merge
density_df = density_df.merge(vcr_2019_2023_exito[['NombreProveedorLimpio', 'RubroPotencial','Exito']],
                              left_on=['NombreProveedorLimpio', 'RubroPotencial'],
                              right_on=['NombreProveedorLimpio', 'RubroPotencial'],
                              how='left')

In [ ]:
density_df.columns

In [ ]:
# Reemplazar NaN en 'Exito' por 0 (proveedores que no obtuvieron ventaja comparativa en el rubro potencial)
density_df['Exito'] = density_df['Exito'].fillna(0).astype(int)

In [ ]:
# Verificar la distribución de la etiqueta
print(density_df['Exito'].value_counts())

In [ ]:
density_df.shape

In [ ]:
#pip install pyarrow

In [ ]:
density_df.to_parquet("density_data.parquet", engine="pyarrow", compression="snappy")

In [ ]:
# Para leer el archivo Parquet que guardaste anteriormente
density_df = pd.read_parquet("density_data.parquet")
density_df.shape

## 11. Definir las Características (Variables Predictoras) y la Etiqueta

In [ ]:
def reduce_memory_usage(df):
    """Reduce el uso de memoria de un DataFrame."""
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:  # Excluir columnas categóricas
            if str(col_type).startswith('int'):
                df[col] = pd.to_numeric(df[col], downcast='integer')
            elif str(col_type).startswith('float'):
                df[col] = pd.to_numeric(df[col], downcast='float')
    return df

# Reducir el uso de memoria
density_df = reduce_memory_usage(density_df)

# Verificar la reducción de memoria
print(density_df.info())

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             roc_auc_score, average_precision_score, confusion_matrix)
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

# Variables predictoras y objetivo
X = density_df.drop(['RubroPotencial', 'NombreProveedorLimpio', 'Exito'], axis=1)
y = density_df['Exito']

# Escalar las variables predictoras
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convertir X_scaled nuevamente a un DataFrame con los nombres originales
X = pd.DataFrame(X_scaled, columns=X.columns)

# Reemplazar valores NaN por 0
X = X.fillna(0)

## 12. Implementar Modelo XGboost con proporción 7 veces mas de clase mayoritaria y evaluar en la distribución real de los datos.

In [ ]:
# Dividir los datos en entrenamiento y prueba
X_train_original, X_test, y_train_original, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             roc_auc_score, average_precision_score, confusion_matrix, 
                             roc_curve, precision_recall_curve)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import xgboost as xgb
from sklearn.utils import resample

# Crear un conjunto balanceado (Proporción 7)
data_train = X_train_original.copy()
data_train['Exito'] = y_train_original

# Separar las clases
data_majority = data_train[data_train['Exito'] == 0]
data_minority = data_train[data_train['Exito'] == 1]

# Submuestrear la clase mayoritaria
proportion = 7
data_majority_downsampled = resample(
    data_majority,
    replace=False,  # Sin reemplazo
    n_samples=len(data_minority) * proportion,
    random_state=42
)

# Combinar las clases balanceadas
data_balanced = pd.concat([data_majority_downsampled, data_minority])

# Separar las variables predictoras (X) y la etiqueta (y)
X_train = data_balanced.drop('Exito', axis=1)
y_train = data_balanced['Exito']

# Entrenar el modelo
model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)
model.fit(X_train, y_train)

# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)
auc_pr = average_precision_score(y_test, y_pred_proba)

# Obtener la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Visualizar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Exito', 'Exito'], yticklabels=['No Exito', 'Exito'])
plt.title('Matriz de Confusión')
plt.xlabel('Predicciones')
plt.ylabel('Valores Reales')
plt.show()

# Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC-ROC = {auc_roc:.4f}")
plt.plot([0, 1], [0, 1], 'k--', label="Random")
plt.title('Curva ROC')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.legend()
plt.grid()
plt.show()

# Curva PR
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(recall_vals, precision_vals, label=f"AUC-PR = {auc_pr:.4f}")
plt.title('Curva de Precisión-Recall')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.grid()
plt.show()

# Obtener la importancia de las variables
importance = model.get_booster().get_score(importance_type='weight')

# Garantizar que todas las variables se incluyan, incluso las que no fueron usadas (importancia 0)
all_features = X_train.columns.tolist()
importance_full = {feature: importance.get(feature, 0) for feature in all_features}

# Convertir a DataFrame
importance_df = pd.DataFrame({
    'Feature': list(importance_full.keys()),
    'F_Score': list(importance_full.values())
})
importance_df['Percentage'] = (importance_df['F_Score'] / importance_df['F_Score'].sum()) * 100
importance_df = importance_df.sort_values(by='Percentage', ascending=False)

# Mostrar resultados
def print_results():
    print("\n### Resultados del Modelo ###")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC-ROC: {auc_roc:.4f}")
    print(f"AUC-PR: {auc_pr:.4f}")

    print("\n### Importancia de Variables (Todas) ###")
    print(importance_df)

# Visualización opcional de todas las variables con colores diferenciados
plt.figure(figsize=(12, 8))

# Asignar colores: 'red' para "density relatedness" y 'skyblue' para las demás
colors = ['red' if feature == 'density relatedness' else 'skyblue' for feature in importance_df['Feature']]

plt.barh(importance_df['Feature'], importance_df['Percentage'], color=colors)
plt.xlabel('Porcentaje de Importancia')
plt.ylabel('Variable')
plt.title('Importancia de Todas las Variables')
plt.gca().invert_yaxis()
plt.show()

# Call to display results
print_results()


In [ ]:
# Asignar colores: rojo para "density relatedness" y gris para las demás
colors = ['red' if feature == 'DensityRelatedness' else 'lightgray' for feature in importance_df['Feature']]

# Gráfico
plt.figure(figsize=(12, 8))
plt.barh(importance_df['Feature'], importance_df['Percentage'], color=colors, edgecolor='none')

# Configuración del gráfico
plt.xlabel('Porcentaje de Importancia', fontsize=14)
plt.ylabel('Variable', fontsize=14)
plt.title('Importancia de Variables (Destacando density relatedness)', fontsize=16)
plt.gca().invert_yaxis()  # Invertir el eje y para que las variables con mayor importancia estén arriba
plt.show()

### 12.1 Análisis de correlación

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Variables predictoras completas
features = [
    'TotalMontoAdjudicadoProveedor', 'DensityRelatedness',
    'TotalMontoPromedioAdjudicadoProveedor', 'NumeroRubrosConVentaja',
    'TotalParticipacionesProveedor', '%AdjudicacionProveedor',
    'NumeroProveedoresRubro', 'TotalAdjudicacionesProveedor',
    'TotalMontoAdjudicadoRubro', 'TotalMontoPromedioAdjudicadoRubro',
    'weighted_degree', 'clustering_coefficient','betweenness_centrality', 
    'pagerank', 'closeness_centrality']


# Matriz de correlación
correlation_matrix = X_train_original[features].corr()

# Visualizar matriz de correlación
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlación de Variables Predictoras")
plt.show()

### 12.2 Analisis Multicolinealidad

Eliminar las variables con VIF extremadamente alto como:

- weighted_degree (282.01)
- clustering_coefficient (42.09)
- pagerank (207.85)
- eigenvector_centrality (32.83)

Estas variables presentan una alta multicolinealidad, lo que significa que están explicando información que ya está capturada por otras variables en el modelo.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Crear un DataFrame con las variables predictoras
X_vif = X_train_original[features]

# Calcular VIF
vif_data = pd.DataFrame()
vif_data["Variable"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

print("\n### VIF de las Variables Predictoras ###")
print(vif_data)

In [ ]:
#pip install xgboost

In [ ]:
# Importar la clase StandardScaler
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Variables predictoras (excluyendo 'RubroPotencial', 'NombreProveedorLimpio', 'Exito', etc.)
X = density_df.drop(['RubroPotencial', 'NombreProveedorLimpio', 'Exito','pagerank'], axis=1)

# Crear un escalador
scaler = StandardScaler()

# Ajustar y transformar X
X_scaled = scaler.fit_transform(X)

# Convertir X_scaled de nuevo a un DataFrame y asignar los nombres de las columnas
X = pd.DataFrame(X_scaled, columns=X.columns)

# Etiqueta (variable objetivo)
y = density_df['Exito']

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample

# Dividir los datos originales en entrenamiento y prueba (proporción original)
X_train_original, X_test, y_train_original, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
from sklearn.utils import resample
import pandas as pd
import xgboost as xgb

# Definir los datos originales (asegúrate de que `X_train_original` y `y_train_original` estén definidos)
data_train = X_train_original.copy()
data_train['Exito'] = y_train_original

# Separar las clases
data_majority = data_train[data_train['Exito'] == 0]
data_minority = data_train[data_train['Exito'] == 1]

# Proporción específica: 7
proportion = 7

# Submuestrear la clase mayoritaria
data_majority_downsampled = resample(
    data_majority,
    replace=False,  # Sin reemplazo
    n_samples=len(data_minority) * proportion,
    random_state=42
)

# Combinar la clase minoritaria con la clase mayoritaria submuestreada
data_balanced = pd.concat([data_majority_downsampled, data_minority])

# Separar X y y en el conjunto balanceado
X_train = data_balanced.drop('Exito', axis=1)
y_train = data_balanced['Exito']

# Crear y entrenar el modelo
model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)
model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calcular métricas principales
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Calcular AUC-ROC y AUC-PR
auc_roc = roc_auc_score(y_test, y_pred_proba)
auc_pr = average_precision_score(y_test, y_pred_proba)

# Guardar la importancia de las variables
importance = model.get_booster().get_score(importance_type='weight')
importance_df = pd.DataFrame({
    'Feature': list(importance.keys()),
    'F_Score': list(importance.values())
})
importance_df['Percentage'] = (importance_df['F_Score'] / importance_df['F_Score'].sum()) * 100
importance_df = importance_df.sort_values(by='Percentage', ascending=False)

# Mostrar los resultados
print("\n### Resultados del Modelo 5 (Proporción 7) ###")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")
print(f"AUC-PR: {auc_pr:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nFeature Importance:")
print(importance_df)

# Opcional: Guardar los resultados en un archivo CSV
#importance_df.to_csv("feature_importance_proportion_7.csv", index=False)


In [ ]:
c]

## 14. Densi

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Asegúrate de que la columna 'DensityRelatedness' y 'Exito' sean numéricas
density_df['DensityRelatedness'] = pd.to_numeric(density_df['DensityRelatedness'], errors='coerce')
density_df['Exito'] = pd.to_numeric(density_df['Exito'], errors='coerce')

# Dividir los valores de Density Relatedness en bins
bins = np.linspace(0, 1, 11)  # Dividir de 0 a 1 en 10 intervalos iguales
density_df['Density_Bin'] = pd.cut(density_df['DensityRelatedness'], bins=bins)

# Calcular la probabilidad promedio de éxito por bin y el error estándar
bin_means = density_df.groupby('Density_Bin')['Exito'].mean()
bin_errors = density_df.groupby('Density_Bin')['Exito'].sem()  # Error estándar de la media

# Etiquetas de los bins
bin_labels = [f'{round(b.left, 2)} - {round(b.right, 2)}' for b in bin_means.index]

# Graficar
plt.figure(figsize=(10, 6))
plt.bar(bin_labels, bin_means, yerr=bin_errors, alpha=0.7, capsize=5, color='skyblue')
plt.xlabel('Density Relatedness (ωpr)')
plt.ylabel('Probabilidad de desarrollar VCR en 5 años')
plt.title('Probabilidad de desarrollar VCR por Density Relatedness')
plt.xticks(rotation=45)  # Rotar etiquetas para mejor visibilidad
plt.tight_layout()  # Ajustar los márgenes
plt.show()


## 15. Análisis del MontoAdjudicado

In [ ]:
# Optimizar tipos de datos
lic_all_years_clean_2014_2018['MontoLineaAdjudica'] = lic_all_years_clean_2014_2018['MontoLineaAdjudica'].astype(np.float32)
mapeo_oferta = {
    'Seleccionada': 'Seleccionada',
    'No Seleccionada': 'No Seleccionada',
    'No sleccionada': 'No Seleccionada',
    'Perdedora': 'No Seleccionada'
}
lic_all_years_clean_2014_2018['Oferta seleccionada'] = lic_all_years_clean_2014_2018['Oferta seleccionada'].replace(mapeo_oferta)
# Mapear a valores numéricos para cálculos
lic_all_years_clean_2014_2018['Oferta seleccionada'] = lic_all_years_clean_2014_2018['Oferta seleccionada'].map({'Seleccionada': 1, 'No Seleccionada': 0}).astype(np.int8)

In [ ]:
# Verifica que 'Oferta seleccionada' exista, si no, créala
if 'Oferta seleccionada' not in lic_all_years_clean_2014_2018.columns:
    lic_all_years_clean_2014_2018['Oferta seleccionada'] = 'No seleccionada'

# Modifica los valores según la condición
lic_all_years_clean_2014_2018['Oferta seleccionada'] = lic_all_years_clean_2014_2018['MontoLineaAdjudica'].apply(
    lambda x: 'Seleccionada' if x > 0 else 'No seleccionada'
)

In [ ]:
lic_all_years_clean_2014_2018['Oferta seleccionada'].value_counts()

In [ ]:
# Filtrar valores válidos (mayores a 1 y menores a 10**12)
valid_data = lic_all_years_clean_2014_2018[
    (lic_all_years_clean_2014_2018['MontoLineaAdjudica'] > 1) & 
    (lic_all_years_clean_2014_2018['MontoLineaAdjudica'] < 10**12)
]

# Verifica el resultado del filtrado
print(f"Número de registros después del filtrado: {len(valid_data)}")
print("Resumen estadístico de MontoLineaAdjudica (post-filtrado):")
print(valid_data['MontoLineaAdjudica'].describe())

In [ ]:
# Visualización inicial de la distribución
plt.figure(figsize=(10, 6))
plt.hist(valid_data['MontoLineaAdjudica'], bins=50, edgecolor='black')
plt.title('Distribución inicial de MontoLineaAdjudica (valores válidos)')
plt.xlabel('Monto')
plt.ylabel('Frecuencia')
plt.yscale('log')  # Escala logarítmica para exponer valores más altos
plt.show()

In [ ]:
valid_data.head()

## 16. Visualizando un caso de éxito 

In [ ]:
import networkx as nx
    import matplotlib.pyplot as plt

    # Filtrar portafolio actual (rubros con VCR > 1)
    proveedor = "soloverde"
    portafolio_actual = vcr_df[(vcr_df['NombreProveedorLimpio'] == proveedor) & (vcr_df['VCR'] > 1)]

    # Identificar rubros potenciales desde density_df
    rubros_potenciales = density_df[density_df['NombreProveedorLimpio'] == proveedor]

    # Seleccionar 2 rubros exitosos y 2 no exitosos
    rubros_exitosos = rubros_potenciales[rubros_potenciales['Exito'] == True].head(2)
    rubros_no_exitosos = rubros_potenciales[rubros_potenciales['Exito'] == False].head(2)

    # Combinar todos los nodos (actuales y potenciales)
    nodos_actuales = portafolio_actual['RubroN2'].tolist()
    nodos_exitosos = rubros_exitosos['RubroPotencial'].tolist()
    nodos_no_exitosos = rubros_no_exitosos['RubroPotencial'].tolist()
    todos_nodos = nodos_actuales + nodos_exitosos + nodos_no_exitosos

    # Crear el grafo desde proximidad_df filtrando nodos relevantes
    G = nx.Graph()
    for nodo in todos_nodos:
        G.add_node(nodo, tipo='actual' if nodo in nodos_actuales else 'exitoso' if nodo in nodos_exitosos else 'no_exitoso')

    for i, nodo1 in enumerate(todos_nodos):
        for nodo2 in todos_nodos[i + 1:]:
            if nodo1 in proximidad_df.index and nodo2 in proximidad_df.columns:
                peso = proximidad_df.loc[nodo1, nodo2]
                if peso > 0:  # Filtrar solo conexiones relevantes
                    G.add_edge(nodo1, nodo2, weight=peso)

    # Asignar colores mejorados a los nodos
    color_map_actuales = '#1f78b4'  # Azul intenso
    color_map_exitosos = '#33a02c'  # Verde esmeralda
    color_map_no_exitosos = '#ff7f00'  # Naranja brillante
    node_colors = [
        color_map_actuales if G.nodes[n]['tipo'] == 'actual' 
        else color_map_exitosos if G.nodes[n]['tipo'] == 'exitoso' 
        else color_map_no_exitosos 
        for n in G.nodes
    ]

    # Dibujar el grafo
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G, seed=42, k=0.5)  # Ajustar posiciones
    nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color='gray', node_size=800, font_size=9, alpha=0.9)

    # Agregar leyenda
    legend_labels = {
        color_map_actuales: 'Rubros Actuales',
        color_map_exitosos: 'Rubros Potenciales Exitosos',
        color_map_no_exitosos: 'Rubros Potenciales No Exitosos'
    }
    for color, label in legend_labels.items():
        plt.scatter([], [], color=color, label=label)
    plt.legend(loc='best')

    plt.title(f'Portafolio y Rubros Potenciales del Proveedor: {proveedor}')
    plt.show()